<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/datacollection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import json
import pytz
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime 
from datetime import timedelta
import pickle
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/[Shared]K-Project/data/'

Mounted at /content/drive


In [2]:
## Load pickle - fullcode 목록
with open(data_path + 'full_code.pkl',"rb") as f: 
    fullcode = pickle.load(f)
    
# 상장폐지 상태 제외시키기
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']

# fullcode열을 list로 변경
fullcode_list = list(fullcode_filter['full_code'])
print(fullcode_list[:10])
print(len(fullcode_list))

['KR7060310000', 'KR7095570008', 'KR7006840003', 'KR7054620000', 'KR7265520007', 'KR7211270004', 'KR7027410000', 'KR7282330000', 'KR7138930003', 'KR7001460005']
2585


In [8]:
# 한국날짜 설정
def kor_date():
    tz = pytz.timezone('Asia/Seoul')  
    raw_dates = datetime.now(tz)
    yes = raw_dates - timedelta(1) 
    yesterday =  yes.strftime("%Y%m%d")  
    return yesterday
a = kor_date()
print(a)

20211007


In [14]:
# 데이터 가져와서 필요 컬럼 추출 후 데이터프레임 생성하여 json으로 저장하는 함수
def make_initialdata(code):
    global result
    
    data2 = { 'bld': 'dbms/MDC/STAT/issue/MDCSTAT23902',
    'isuCd': code, 
    'isuCd2': '',
    'strtDd': strtDd,
    'endDd': endDd,
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data2, headers)
    j2 = json.loads(r2.text)
    df2 = pd.DataFrame(j2['output'])
    # print(df2)

    data3 = { 'bld': 'dbms/MDC/STAT/standard/MDCSTAT03502',
    'isuCd': code,
    'isuCd2': code,
    'strtDd': strtDd,
    'endDd': endDd,
    'searchType': '2',
    'mktId': 'ALL',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data3, headers)
    j2 = json.loads(r2.text)
    df3 = pd.DataFrame(j2['output'])
    # print(df3)

    if len(df2.columns) > 0: # df2에 column이 존재하면 (df가 비어서 에러나는 경우 방지)
        df2_ = df2[['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP']]
    else: # df가 비었을때 column 생성해줌
        df2_ = pd.DataFrame(index = range(0,len(df3)),
                           columns = ['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP'])

    if len(df3.columns) > 0:
        df3_ = df3[['EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD']]
    else:
        df3_ = pd.DataFrame(index = range(0,len(df2)),
                           columns = ['EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD'])
 
    result = pd.concat([df2_, df3_], axis = 1)
    print(result)

    # result_js = result.to_json(orient = 'columns') # 데이터프레임을 json 형태로 변환
    # result_dict[i] = result_js # fullcode가 key값이 되고 json이 value값이 됨
    # print(len(result_dict)) # 현재 상황을 알기위해 몇개의 종목이 저장되었는지 갯수 확인

In [17]:
url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
headers = {'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',}

result_dict = {}
for i in fullcode_list[:1]: # 테스트를 위해 1개만 실행
    print('fullcode:',i)
    with open(data_path + 'initial_data1.json',"r") as json_file:
        json_data = json.load(json_file) # json file open

    df = pd.read_json(json_data[i], orient ='index') # 첫번째 키값으로 데이터프레임 변환
    json_df = df.transpose()
    print('원래있던 데이터--------------------------------------------------------')
    print(json_df)

    date = json_df.iloc[0,0] # 수집된 데이터의 마지막 날짜 확인
    yesterday = re.sub('/','-',date)
    yesterday = datetime.strptime(yesterday, '%Y-%m-%d') + timedelta(days = 1)
    yesterday = yesterday.strftime("%Y%m%d")

    strtDd = yesterday # 수집된 데이터의 마지막 날짜
    endDd = kor_date() # 10월 8일 00:30 코드실행이라면 10월 7일까지의 데이터 가져와야 함
    print('strtDd: ', strtDd)
    print('endDd: ', endDd)

    print('새로 수집된 데이터------------------------------------------------------')
    make_initialdata(i)
    
    a = pd.concat([result, json_df], axis = 0)
    print('최종데이터 -----------------------------------------------------------')
    print(a)
    
# # 최종 결과물 저장   
# with open('/content/drive/MyDrive/k-project/output/initial_data.json', 'w') as f:
#     json.dump(result_dict, f)

fullcode: KR7060310000
원래있던 데이터--------------------------------------------------------
          TRD_DD  ISU_CD  ISU_NM TDD_CLSPRC  ...  BPS   PBR DPS DVD_YLD
0     2021/10/06  060310      3S      3,615  ...  792  4.56   0    0.00
1     2021/10/05  060310      3S      3,870  ...  792  4.89   0    0.00
2     2021/10/01  060310      3S      3,960  ...  792  5.00   0    0.00
3     2021/09/30  060310      3S      4,105  ...  792  5.18   0    0.00
4     2021/09/29  060310      3S      4,160  ...  792  5.25   0    0.00
...          ...     ...     ...        ...  ...  ...   ...  ..     ...
4806  2002/04/29  060310  삼에스코리아      6,780  ...    -     -   -       -
4807  2002/04/26  060310  삼에스코리아      7,700  ...    -     -   -       -
4808  2002/04/25  060310  삼에스코리아      8,750  ...    -     -   -       -
4809  2002/04/24  060310  삼에스코리아      8,510  ...    -     -   -       -
4810  2002/04/23  060310  삼에스코리아      7,600  ...    -     -   -       -

[4811 rows x 14 columns]
strtDd:  20211007
endD